In [ ]:
import bs4 #解析HTML

from langsmith import Client #導入別人pronmpt模板
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader #發送 HTTP 請求
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_ollama import ChatOllama

from langchain_huggingface.embeddings import HuggingFaceEmbeddings


##indexing##

#load documents

loader=WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/"),

    #只抓標題,作者,正文
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),

)

docs=loader.load()

#spilit

#用遞迴式切割，每個 chunk 最多 chunk_size個token先嘗試用較大的語意邊界切（例如：段落 \n\n）
#相鄰 chunk 會重疊 chunk_overlap 個token避免關鍵資訊被切斷
text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300,chunk_overlap=50)
splits=text_splitter.split_documents(docs)

#embed
embedding_model =HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-zh-v1.5",

)

vectorstore=Chroma.from_documents(documents=splits,embedding=embedding_model)
retriever=vectorstore.as_retriever()

#prompt

#調用LangChain Hub上面別人的模板

client = Client()
prompt = client.pull_prompt("rlm/rag-prompt")

#LLM

llm=ChatOllama(model="llama3.2:3b-instruct-q8_0",temperature=0)

# Post-processing 把retriever的page_content訊息拼接好給llm
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    RunnableParallel(
        {
            "context": retriever | format_docs,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("What is task decomposition for LLM agents?")

c:\Users\user\anaconda3\envs\torch-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


'Task decomposition for LLM agents involves breaking down large tasks into smaller, manageable subgoals to enable efficient handling of complex tasks. This can be done through simple prompting, task-specific instructions, or human input. The goal is to transform big tasks into multiple simpler steps that the LLM can handle more efficiently.'